# Aggregating DataFrames
In this notebook, you’ll calculate summary statistics on DataFrame columns, and master grouped summary statistics and pivot tables.

#### Import libraries

In [ ]:
import pandas as pd
import numpy as np

#### Import data

In [ ]:
# Read CSV into a pandas dataframe
sales = pd.read_csv('../../data/walmart.csv', index_col=0)

# converts date column to a datetime object
sales['date'] = pd.to_datetime(sales['date'].values)

***
<br>

# Summary statistics

# Mean and median
Summary statistics are exactly what they sound like - they summarize many numbers in one statistic. For example, mean, median, minimum, maximum, and standard deviation are summary statistics. Calculating summary statistics allows you to get a better sense of your data, even if there's a lot of it.
<br>
<br>
##### Instructions
- Explore your new DataFrame first by viewing the first few rows of the sales DataFrame.
- View information about the columns in `sales`.
- View the mean of the `weekly_sales` column.
- View the median of the `weekly_sales` column.

In [ ]:
# View the head of the sales DataFrame
sales.head()

In [ ]:
# View info about the sales DataFrame
sales.info()

In [ ]:
# View the mean of weekly_sales
sales['weekly_sales'].mean()

In [ ]:
# Print the median of weekly_sales
sales['weekly_sales'].median()

The mean weekly sales amount is almost double the median weekly sales amount! This can tell you that there are a few very high sales weeks that are making the mean so much higher than the median.

# Summarizing dates
Summary statistics can also be calculated on date columns that have values with the data type `datetime64`. Some summary statistics — like mean — don't make a ton of sense on dates, but others are super helpful, for example, minimum and maximum, which allow you to see what time range your data covers.
<br>
<br>
##### Instructions
- View the maximum of the `date` column.
- View the minimum of the `date` column.

In [ ]:
# View the maximum of the date column
sales['date'].max()

In [ ]:
# View the minimum of the date column
sales['date'].min()

Taking the minimum and maximum of a column of dates is handy for figuring out what time period your data covers.

# Efficient summaries
While pandas and NumPy have tons of functions, sometimes, you may need a different function to summarize your data.

The `.agg()` method allows you to apply your own custom functions to a DataFrame, as well as apply functions to more than one column of a DataFrame at once, making your aggregations super-efficient. For example: `df['column'].agg(function)`

In the custom function for this exercise, the inter quartile range, is the 75th percentile minus the 25th percentile. It's an alternative to standard deviation that is helpful if your data contains outliers.

In [ ]:
# A custom inter-quartile range function
def inter_quartile_range(column):
    return column.quantile(0.75) - column.quantile(0.25)

##### Instructions 1/3
- Use the custom `inter_quartile_range()` function defined for you along with `.agg()` to view the inter quartile range of the `temperature_c` column of `sales`.

In [ ]:
# View the inter_quartile_range of the temperature_c column
sales['temperature_c'].agg(inter_quartile_range)

##### Instructions 2/3
Update the column selection to use the custom `inter_quartile_range()` function with `.agg()` to view the inter quartile range of `temperature_c`, `fuel_price_usd_per_l`, and `unemployment`, in that order.


In [ ]:
# Update to view inter_quartile_range of temperature_c, fuel_price_usd_per_l, & unemployment
sales[["temperature_c", "fuel_price_usd_per_l", 'unemployment']].agg(inter_quartile_range)

##### Instructions 3/3
- Update the aggregation functions called by `.agg()`: include `inter_quartile_range` and `np.median` in that order.

In [ ]:
# Update to view inter_quartile_range and median of temperature_c, fuel_price_usd_per_l, & unemployment
sales[["temperature_c", "fuel_price_usd_per_l", "unemployment"]].agg([inter_quartile_range, np.median])

The `.agg()` method makes it easy to compute multiple statistics on multiple columns, all in just one line of code.

# Cumulative statistics
Cumulative statistics can also be helpful in tracking summary statistics over time. In this section, you'll calculate the cumulative sum and cumulative max of a department's weekly sales, which will allow you to identify what the total sales were so far as well as what the highest weekly sales were so far.
<br>
<br>
##### Instructions
1. Create a DataFrame called `sales_1_1`, which contains the sales data for department 1 of store 1.
2. Sort the rows of `sales_1_1` by the `date` column in ascending order.
3. Get the cumulative sum of `weekly_sales` and add it as a new column of `sales_1_1` called `cum_weekly_sales`.
4. Get the cumulative maximum of `weekly_sales`, and add it as a column called `cum_max_sales`.
5. View the `date`, `weekly_sales`, `cum_weekly_sales`, and `cum_max_sales` columns.

In [ ]:
# Create a DataFrame `sales_1_1`, which contains the sales data for department 1 of store 1
sales_1_1 = sales[(sales['department'] == 1) & (sales['store'] == 1)]

# Sort sales_1_1 by date
sales_1_1 = sales_1_1.sort_values('date', ascending=True)

# Get the cumulative sum of weekly_sales, add as cum_weekly_sales col
sales_1_1['cum_weekly_sales'] = sales_1_1['weekly_sales'].cumsum()

# Get the cumulative max of weekly_sales, add as cum_max_sales col
sales_1_1['cum_max_sales'] = sales_1_1['weekly_sales'].cummax()

# See the columns you calculated
sales_1_1[["date", "weekly_sales", "cum_weekly_sales", "cum_max_sales"]]

Not all functions that calculate on columns return a single number. Some, like the cumulative statistic functions, return a whole column.

***
<br>

# Counting

# Dropping duplicates
Removing duplicates is an essential skill to get accurate counts because often, you don't want to count the same thing multiple times. In this section, you'll create some new DataFrames using unique values from `sales`.
<br>
<br>
##### Instructions
1. Remove rows of `sales` with duplicate pairs of `store` and `type` and save as `store_types` and view the head.
2. Remove rows of `sales` with duplicate pairs of `store` and `department` and save as `store_depts` and view the head.
3. Subset the rows that are holiday weeks using the `is_holiday` column, and drop the duplicate `date`s, saving as `holiday_dates`.
4. Select the `date` column of `holiday_dates`, and view.

In [ ]:
# Drop duplicate store/type combinations
store_types = sales.drop_duplicates(subset=['store', 'type'])
store_types.head()

In [ ]:
# Drop duplicate store/department combinations
store_depts = sales.drop_duplicates(subset=['store', 'department'])
store_depts.head()

In [ ]:
# Subset the rows where is_holiday is True and drop duplicate dates
holiday_dates = sales[sales['is_holiday']].drop_duplicates(subset=['date'])

# View date col of holiday_dates
holiday_dates['date']

The holiday weeks correspond to the Superbowl in February, Labor Day in September, Thanksgiving in November, and Christmas in December. Now that the duplicates are removed, it's time to do some counting.

# Counting categorical variables
Counting is a great way to get an overview of your data and to spot curiosities that you might not notice otherwise. In this section, you'll count the number of each type of store and the number of each department number using the DataFrames `store_types` and `store_depts` from the previous section.
<br>
<br>
##### Instructions
1. Count the number of stores of each store `type` in `store_types`.
2. Count the proportion of stores of each store `type` in `store_types`.
3. Count the number of different `departments` in `store_depts`, sorting the counts in descending order.
4. Count the proportion of different `department`s in `store_depts`, sorting the proportions in descending order.

In [ ]:
# Count the number of stores of each type
store_counts = store_types['type'].value_counts()
store_counts

In [ ]:
# Get the proportion of stores of each type
store_proportions = store_types['type'].value_counts(normalize=True)
store_proportions

In [ ]:
# Count the number of each department number and sort
dept_counts_sorted = store_depts['department'].value_counts(sort=True)
dept_counts_sorted

In [ ]:
# Get the proportion of departments of each number and sort
dept_props_sorted = store_depts['department'].value_counts(sort=True, normalize=True)
dept_props_sorted

***
<br>

# Grouped summary statistics

# What percent of sales occurred at each store type?
While `.groupby()` is useful, you can calculate grouped summary statistics without it.

Walmart distinguishes three types of stores: "supercenters," "discount stores," and "neighborhood markets," encoded in this dataset as type "A," "B," and "C." In this section , you'll calculate the total sales made at each store type, without using `.groupby()`. You can then use these numbers to see what proportion of Walmart's total sales were made at each type.

<br>

##### Instructions
1. Calculate the total `weekly_sales` over the whole dataset.
2. Subset for type `A` stores, and calculate their total weekly sales. Do the same for `type` `B` and type `C` stores.
3. Combine the A/B/C results into a list, and divide by `sales_all` to get the proportion of sales by type.

In [ ]:
# Calc total weekly sales
sales_all = sales["weekly_sales"].sum()

# Subset for type A stores, calc total weekly sales
sales_A = sales[sales["type"] == "A"]["weekly_sales"].sum()
# Subset for type B stores, calc total weekly sales
sales_B = sales[sales["type"] == "B"]["weekly_sales"].sum()
# Subset for type C stores, calc total weekly sales
sales_C = sales[sales["type"] == "C"]["weekly_sales"].sum()

# Get proportion for each type
sales_propn_by_type = [sales_A, sales_B, sales_C] / sales_all
sales_propn_by_type

About 91% of sales occurred in stores of type A', 9% in stores of type B, and there are no sales records for stores of type C.

# Calculations with .groupby()
The `.groupby()` method makes life much easier. In this section, you'll perform the same calculations as last time, except you'll use the `.groupby()` method. You'll also perform calculations on data grouped by two variables to see if sales differ by store type depending on if it's a holiday week or not.
<br>
<br>
##### Instructions 1/2
1. Group `sales` by `type`, take the sum of `weekly_sales`, and store as `sales_by_type`.
2. Calculate the proportion of sales at each store type by dividing by the sum of `sales_by_type`. Assign to `sales_propn_by_type`.

In [ ]:
# Group by type; calc total weekly sales
sales_by_type = sales.groupby("type")["weekly_sales"].sum()
sales_by_type

In [ ]:
# Get proportion for each type
sales_propn_by_type =  sales_by_type / sum(sales_by_type)
sales_propn_by_type

##### Instructions 2/2
- Group sales by `type` and `is_holiday`, take the sum of `weekly_sales`, and store as `sales_by_type_is_holiday`.

In [ ]:
# Group by type and is_holiday; calc total weekly sales
sales_by_type_is_holiday = sales.groupby(['type', 'is_holiday'])['weekly_sales'].sum()
sales_by_type_is_holiday

# Multiple grouped summaries
Earlier in this chapter, you saw that the `.agg()` method is useful to compute multiple statistics on multiple variables. It also works with grouped data. NumPy, which is imported as `np`, has many summary statistics functions, including: `np.min`, `np.max`, `np.mean`, and `np.median`.

<br>

##### Instructions
1. Get the min, max, mean, and median of `weekly_sales` for each store type using `.groupby()` and `.agg()`. Store this as `sales_stats`. Make sure to use `numpy` functions!
2. Get the min, max, mean, and median of `unemployment` and `fuel_price_usd_per_l` for each store type. Store this as `unemp_fuel_stats`.

In [ ]:
# For each store type, aggregate weekly_sales: get min, max, mean, and median
sales_stats = sales.groupby('type')['weekly_sales'].agg([np.min, np.max, np.mean, np.median])
sales_stats

Notice that the minimum `weekly_sales` is negative because some stores had more returns than sales.

In [ ]:
# For each store type, aggregate unemployment and fuel_price_usd_per_l: get min, max, mean, and median
unemp_fuel_stats = sales.groupby('type')[['unemployment', 'fuel_price_usd_per_l']].agg([np.min, np.max, np.mean, np.median])
unemp_fuel_stats

***
<br>

# Pivot tables

# Pivoting on one variable
Pivot tables are the standard way of aggregating data in spreadsheets. In pandas, pivot tables are essentially just another way of performing grouped calculations. That is, the `.pivot_table()` method is just an alternative to `.groupby()`.

In this section, you'll perform calculations using `.pivot_table()` to replicate the calculations you performed in the last section using `.groupby()`.
<br>
<br>
##### Instructions 1/3
Get the mean `weekly_sales` by `type` using `.pivot_table()` and store as `mean_sales_by_type`.

In [ ]:
# Pivot for mean weekly_sales for each store type
mean_sales_by_type = sales.pivot_table(values='weekly_sales', index='type')

# View mean_sales_by_type
mean_sales_by_type

##### Instructions 2/3
Get the mean and median (using NumPy functions) of `weekly_sales` by `type` using `.pivot_table()` and store as `mean_median_sales_by_type`.

In [ ]:
# Pivot for mean and median weekly_sales for each store type
mean_median_sales_by_type = sales.pivot_table(values='weekly_sales',
                                              index='type',
                                              aggfunc=[np.mean, np.median])

# View mean_med_sales_by_type
mean_median_sales_by_type

##### Instructions 3/3
Get the mean of `weekly_sales` by `type` and `is_holiday` using `.pivot_table()` and store as `mean_sales_by_type_holiday`.

In [ ]:
# Pivot for mean weekly_sales by store type and holiday
mean_sales_by_type_holiday = sales.pivot_table(values="weekly_sales",
                                               index="type",
                                               columns="is_holiday")

# View mean_sales_by_type_holiday
mean_sales_by_type_holiday

# Fill in missing values and sum values with pivot tables
The `.pivot_table()` method has several useful arguments, including `fill_value` and `margins`.
- `fill_value` replaces missing values with a real value (known as ***imputation***). What to replace missing values with is a topic big enough to have its own course ([Dealing with Missing Data in Python](https://www.datacamp.com/courses/dealing-with-missing-data-in-python)), but the simplest thing to do is to substitute a dummy value.
- `margins` is a shortcut for when you pivoted by two variables, but also wanted to pivot by each of those variables separately: it gives the row and column totals of the pivot table contents.

In this section, you'll practice using these arguments to up your pivot table skills, which will help you crunch numbers more efficiently!
<br>
<br>

##### Instructions 1/2
View the mean `weekly_sales` by `department` and `type`, filling in any missing values with `0`.

In [ ]:
# View mean weekly_sales by department and type; fill missing values with 0
sales.pivot_table(values='weekly_sales',
                  index='department',
                  columns='type',
                  fill_value=0).head()

##### Instructions 2/2
View the mean `weekly_sales` by `department` and `type`, filling in any missing values with `0` and summing all rows and columns.

In [ ]:
# View the mean weekly_sales by department and type; fill missing values with 0s; sum all rows and cols
sales.pivot_table(values="weekly_sales",
                  index="department",
                  columns="type",
                  fill_value=0,
                  margins=True).head()

You are now armed with pivot table skills that can help you compute summaries at multiple grouped levels in one line of code.